# Monte Carlo Assignment
## The Stokes Flow

##### Laurent Pétré & Ilan Renous

In this assignment, we are going to generate random numbers with different distributions, simulate a brownian motion and simulate a model that predicts magnetism in a ferromagnetic material.

In [ ]:
# We import the libraries we will need
import numpy
from matplotlib import pyplot
%matplotlib inline

## Random numbers

### Built-in Vs DIY Implementations

We are going to generate random numbers following a gaussian and an exponential distibution from a uniform random numbers generator. For the Gaussian distibution we are going to use the Von-Neumann rejection method and compare it to the built in numpy metod.

In [ ]:
N = 100000 # Number of random numbers to draw
h=[] # Empty list for the final set of numbers

# Given Gaussian generator from the course with VNR method
for i in range(N):
    # Distribute g according to the exponential distribution
    u1 = numpy.random.random_sample()
    g = numpy.log(1/(1-u1))
    K = 1.4

    # Generate a second number for the acceptance/rejection condition
    u2 = numpy.random.random_sample()

    # Acceptance/rejection step
    ratio = 2*numpy.exp(-g**2/2.0)/numpy.sqrt(2*numpy.pi) / \
                (K*numpy.exp(-g))
    if (u2<ratio):
        # Append g to the set h
        h.append(g)

# Built-in generator
# We generate twice as much number as with the VNRM 
# to have to have a scaled histogram 
x = numpy.random.normal(loc=0, scale=1, size=2*len(h))

# Plot
pyplot.figure(figsize=(10,10))
pyplot.xlim(-5, 5)

binwidth = 0.2
bins = numpy.arange(-5, 5, binwidth)
pyplot.hist(x, bins, normed=False)
pyplot.hist(h, bins, normed=False, alpha=0.75)

#Expected
bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, \
    numpy.exp(-bin_centers**2/2.0)/numpy.sqrt(2*numpy.pi))

We see that the two distribution give very similar random number distribution.

For the exponential distribution, we use the inverse function method. This method tells us that the function $$
F(x) = -\frac{1}{\alpha}\log(1-x).
$$
follows a exponential distribution and x a is uniformly distributed in the interval [0.1[

In [ ]:
alpha = 0.25 # Rate of the exponential distribution

# Built-in generator
x = numpy.random.exponential(scale=1/alpha, size=100000)

# Inverse function
u = numpy.random.random_sample(100000)
y = -1/alpha*numpy.log(1-u)

# Plot
pyplot.figure(figsize=(10,10))
pyplot.xlim(0, 20)

binwidth = 0.5
bins = numpy.arange(0, 20, binwidth)
pyplot.hist(y, bins, normed=True);
pyplot.hist(x, bins, normed=True, alpha=0.75);

#Expected
bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, alpha*numpy.exp(-alpha*bin_centers))

Once again, we can see that the random numbers genreated by the inversion method are really close to those generated by the built in generator.

### Box-Muller

Now, we are going to use another method to generate a gaussian distribution : the Box-Muller method.

This method can be summurized as follow :

To do somme calculation, we must use the product of two independant gaussian distributions  
$$
f(x, y)=\frac{1}{2\pi} e^{-\frac{x^2+y^2}{2}}
$$

We consider the polar coordinates

$$x=r\cos(\theta)$$
$$y=r\sin(\theta)$$

From the central symmetry, $\theta$ is of course uniformly distributed in the interval $[0, 2\pi[$. With this fact, we can conclude that $r$ is distibuted following the relation:
$$r=\sqrt{-2\ln{u}}$$

Where u is a uniformly distributed variable in the interval $[0,1[$.

In Python that gives us :

In [ ]:
def box_muller(avg, stdev):
    """
    Box Muller implementation.

    Parameters
    ----------
    avg: float
        average of the gauss distribution

    stdev: float
        standard deviation of the gaussian

    Returns
    -------
    array[2]
        two independant numbers distributed following the requested gaussian
    """

    i = numpy.random.random_sample(2)
    r = numpy.sqrt(-2*numpy.log(i[0]))

    return avg + stdev * \
        r*numpy.array([numpy.cos(2*numpy.pi*i[1]), \
                        numpy.sin(2*numpy.pi*i[1])])

# We draw 100000 numbers
x = numpy.empty(100000)
for i in range(0, 50000):
    t = box_muller(0, 1)
    x[i] = t[0]
    x[50000 + i] = t[1]

# Plot
pyplot.xlim(-5, 5)

binwidth = 0.2
bins = numpy.arange(-5, 5, binwidth)
pyplot.hist(x, bins, normed=True);

#Expected
bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, \
        numpy.exp(-bin_centers**2/2.0)/numpy.sqrt(2*numpy.pi))

This method is very efficient and generate, like we can see on this graph, random numbers following a gaussian distribution.

## Brownian motion

The model equation for the evolution of the position of a molecule of dye in water is:

$$
dx=x(t+dt)−x(t)=ds
$$

$ds$ is a displacement resulting from the collision with surrounding water molecules.

We will model $ds$ as
$$ ds=cdt+\sqrt{\alpha dt}\cal{N(0,1)} $$

where $\alpha$ is the same diffusion coefficient that one would use in the diffusion equation. Moreover, at each time step, $\cal{N(0,1)}$ is a random number that is generated from a normal  distribution $f(n)$, centered around $0$, and with variance equal to $1$. 

This is a deplacement forced by a random collision and by a systematic displacement $cdt$. This could model an electrons under a electric field "colliding" randomly with atoms in a metal.

We will follow the path of 100000 particules that are subject to this motion during a time  $T$. At time $t=0$ the positions of the particules are chosen to follow a gaussian distribution. Let's see whats does the position distribution looks like at time $T$.

In [ ]:
# Initial conditions
npart = 100000
mu = 0.
sigma = 1.

# Time to simulate
T = 50.
nt = 500
dt = T/nt

# System parameters
alpha = 2.
c = 0.5

# Generate a set of initial positions based on the Gaussian distribution
x = numpy.random.normal(loc = mu, scale=sigma, size=npart)

# Evolution
for i in range(nt):
    xi = x.copy()
    x = xi + c*dt + numpy.sqrt(2*alpha*dt)*numpy.random.normal(0, 1, npart)

# Plot
pyplot.figure(figsize=(15, 10))
pyplot.xlim(-50, 100)

binwidth = 0.75
bins = numpy.arange(-50, 100, binwidth)
pyplot.hist(x, bins, normed=True);

# Expected
mut = mu + c*T
sigmat2 = 2*alpha*T + sigma**2

bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, \
    numpy.exp(-(bin_centers - mut)**2/(2*sigmat2)) * \
    1/numpy.sqrt(2*numpy.pi*sigmat2))

The particles tend to spread away from each other and are "pushed" towards increasing x.

### Friction

We can model the displacment $ds$ by 
$$ ds=-\gamma x dt +\sqrt{\alpha dt}\cal{N(0,1)} $$

this can be a model for a deplacement forced by a rondom collision with a friction term. We will analize the positions of the particules after a time $T$ with the same initial setup as before.

In [ ]:
# Initial conditions
npart = 100000
mu = 0.
sigma = 1.

# Time to simulate
T = 50
nt = 500
#T1 = 15000
#T2 = 20000

# System parameters
alpha = 2.
gamma = 0.4

def friction(T, nt):
    dt = T/nt

    # Generate a set of initial positions based on the Gaussian distribution
    x = numpy.random.normal(loc = mu, scale=sigma, size=npart)

    # Evolution
    for i in range(nt):
        xi = x.copy()
        x = xi - gamma*xi*dt + \
            numpy.sqrt(2*alpha*dt)*numpy.random.normal(0, 1, npart)

    return x

x = friction(T, nt)
#x1 = friction(T1, nt)
#x2 = friction(T2, nt)

# Plot
pyplot.figure(figsize=(15, 10))
pyplot.xlim(-50, 50)

binwidth = 0.5
bins = numpy.arange(-50, 50, binwidth)
pyplot.hist(x, bins, normed=True);
#pyplot.hist(x1, bins, normed=True);
#pyplot.hist(x2, bins, normed=True, alpha=0.75)

# Expected
mut = mu
sigmat2 = 2*alpha*T + sigma**2 # - f(gamma)

bin_centers = bins[:-1] + binwidth/2
pyplot.plot(bin_centers, \
    numpy.exp(-(bin_centers - mut)**2/(2*sigmat2)) * \
    1/numpy.sqrt(2*numpy.pi*sigmat2))

Here, we compare two distributions at very large time $T1$ and $T2$. We can see that they are really close. We can suppose that the distribution reaches a steady states as $t\rightarrow \infty$. This can be checked anatically:

## Phase transition for ferromagnteic material

Using, the Metropolis algorithm, we are going to compute the magnetisation of a ferromagnetic material following the Ising model. We will prove that a metal, following this model, has different magnetic propreties before and after a temperature $T_c$. This temperature is called the the Curie temperature. 

### Ising model

In the Ising model of ferromagnetism, the material considered is described using dipoles, distributed on a regular lattice and that can either point upwards or downwards (see figure 1). These dipoles represent the atoms that constitute the material and act like magnets oriented in different directions.

<img src="lattice1.png" alt="Drawing" style="width: 300px;"/>
Figure 1

In the simplest version of the Ising model, all the dipoles interact with their nearest neighbors (left, right, above, below). When two neighbors are aligned, the system is in a more stable configuration than when they are aligned in opposite directions. The energy of the system can then be written as,

$$
E=-J\sum_{pairs(i,j)} s_i s_j,
$$

where the sum runs over all the pairs of dipoles in the system and $s_i$ denotes the 'spin' of the $i$-th dipole which is equal to $1$ or $-1$ wether the dipole points upwards or downwards; $J>0$ is the coupling constant for each pair of dipoles. We thus see that the energy is minimized and equal to $-JN_{pairs}$ when all the dipoles point in the same direction. 

In [ ]:
def energy_at_site(sp,alpha,sigma,ix,iy):
    """
    Computes the contribution to the energy for a given spin
    at location ix,iy for a lattice with periodic boundary conditions
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha: real
        coupling constant J/(kb*T)
    sigma: int
        spin at site ix,iy
    ix: int
        location in x
    iy: int
        location in y
   
    Returns:
    -------
    energy: float
        energy for the given configuration
    """

    return -alpha*sigma*(sp[(ix-1)%nx,iy]+sp[(ix+1)%nx,iy]+sp[ix,(iy-1)%ny]+sp[ix,(iy+1)%ny])



Each time a dipole is flipped, the energy changes by an amount equal to,

$$
\Delta E = -{J} s_i \sum_{j \in n(i)} s_j
$$

where $n(i)$ denotes all the neighbors of the $i$-th dipole. $\Delta E$ is positive or negative depending on the sign of $s_i$ and the total spin of the neighbors.



According to statistical mechanics, the probability of finding the system in a given configuration $X$ is equal to,

$$
p(X) = \frac{e^{-\beta E_X}}{Z}
$$

where $E_X$ is the potential energy of the configuration and $\beta=1/k_B T$ where $k_B$ is the so-called Boltzmann constant. In other words, the higher the potential energy, the less likely it is to find the system in the corresponding configuration. In the above formula $Z$ is a normalisation constant such that $\sum_X p(X)=1$,

with $M_X=\sum_{i=1}^N s_i$ for the given state. Above the Curie temperature, one has $<M>=0$ while $<M>$ has a finite non vanishing value below the Curie temperature.

### The Metropolis algorithm

Let us consider an initial state in which each spin takes a random orientation. 
The unit step of the Metropolis algorithm consists in choosing a random spin and in attempting to change its orientation. If the energy of the reversed spin decreases, the flipped state is chosen as the new configuration of the system.
Otherwise if the energy increases by $\Delta E$, the flip is only accepted with the probability,

$$
p_{\rm flip}=e^{-\beta\Delta E},
$$

otherwise the current state is retained as the new configuration. This process is repeated until a large enough number of states $X_k$ has been collected.



In [ ]:
def metropolis_at_site(sp, alpha, ix, iy):
    """
    Flips a dipole at site ix, iy when probability condition is met 
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha  : real
        coupling constant J/(kb*T)
    ix   : int
        location in x
    iy   : int
        location in y
    """

    sigma=sp[ix,iy]
    energy_before_flip = energy_at_site(sp,alpha,sigma,ix,iy)
    sigma = -sigma
    energy_if_site_flipped = energy_at_site(sp,alpha,sigma,ix,iy)
    
    # Flip the site with Metropolis probability
    # Condition is always satisifed if dE < 0
    if (numpy.random.random_sample()<numpy.exp(-(energy_if_site_flipped \
                                               -energy_before_flip))):
        sp[ix,iy]=-sp[ix,iy]

The averages we are interested in, such as the energy or the magnetisation, are then computed according to,

$$
<E> = \frac{\sum_k E_k}{N_k}\\
<M> = \frac{\sum_k M_k}{N_k}
$$

where the sums run over all the states generated through the Metropolis algorithm and $N_k$ is the total number of them.

Compared to the averages over all possible states, we see here that we only sum over a subset of them. However, thanks to the Metropolis algorithm, the states corresponding to different values of the energy are generated with the right distribution function and the averages above converge to the equilibrium values as $N_k\rightarrow \infty$.

In [ ]:
def energy_array(sp, alpha):
    """
    Computes the energy lattice from a spin lattice.
   
    Parameters:
    ----------
    sp: array[NxM]
        array of spins

    alpha: float
        coupling constant
   
    Returns:
    -------
    array[NxM] of float
        energy lattice for the given configuration
    """

    return -alpha*sp[:, :]* \
        (numpy.roll(sp, 1, axis=0) + numpy.roll(sp, -1, axis=0) + \
         numpy.roll(sp, 1, axis=1) + numpy.roll(sp, -1, axis=1))

def total_energy(sp, alpha):
    """
    Computes the total energy of the lattice.
   
    Parameters:
    ----------
    sp: array[NxM]
        array of spins

    alpha: float
        coupling constant

    Returns:
    -------
    float
        total energy for the given configuration
    """

    return energy_array(sp, alpha).sum()

def total_magnetization(sp):
    """
    Computes the total magnetiaztion of the lattice.
   
    Parameters:
    ----------
    sp: array[NxM]
        array of spins
   
    Returns:
    -------
    float
        magnetization for the given configuration
    """

    return sp.sum()

### Implementation

The contribution to the energy coming from a change of spin at one site is computed with the following function. we assume that the system is periodic in both directions :

In [ ]:
def ising_model_metropolis(sp, NMC, nx, ny, alpha):
    """ Creates a sequence of states for the Ising model using
    the Metropolis algorithm
   
    Parameters:
    ----------
    sp   : initial lattice state
    nx   : int
        Discretization points in x
    ny   : int
        Discretization points in y
    NMC  : int
        Number of states to create
    alpha  : real
        coupling constant J/(kb*T)
    Returns:
    -------
    states: sequence of states
    """
    states = numpy.empty([NMC+1,nx,ny])
    states[0] = sp.copy()
    
    for i in range(1,NMC+1):
        for j in range(0,nx*ny):
            ix=numpy.random.random_integers(0,nx-1)
            iy=numpy.random.random_integers(0,ny-1)
            metropolis_at_site(sp,alpha,ix,iy)
        states[i]=sp.copy()
    return states

In [ ]:
NMC = 100
nx = 100
ny = 100

na = 10

alpha = numpy.linspace(0.20, 0.60, na)
#alpha = [0.1, 0.2, 0.3, 0.35, 0.37, 0.39, 0.41, 0.43, 0.45, 0.5, 0.6, 0.7]
energy = numpy.zeros(na)
magn = numpy.zeros(na)

for i in range(0, na):
    sp = numpy.ones([nx,ny])
    states = ising_model_metropolis(sp, NMC, nx, ny, alpha[i])
    for j in range(0, NMC+1):
        energy[i] += total_energy(states[j], alpha[i])/(NMC+1)
        magn[i] += total_magnetization(states[j])/(NMC+1)

In [ ]:
pyplot.figure(figsize=(15, 10))
pyplot.plot(alpha, energy)
pyplot.plot(alpha, magn)

In [ ]:
#secondder_magn = numpy.zeros(na)
#def secondder(f,x):
    #ddf=(1/dx**2)*f[x+1]-2*f[x]+fx[x-1]
    
#secondder_magn=secondder(magn,alpha)

In [ ]:
#pyplot.figure(figsize=(15, 10))
#pyplot.plot(alpha, energy)
#pyplot.plot(alpha,secondder_magn)

##### Source

(1) We used the following lectures https://github.com/numerical-mooc/numerical-mooc available under Creative Commons Attribution license CC-BY 4.0, (c)2014 L.A. Barba, C. Cooper, G.F. Forsyth, A. Krishnan.

---
###### The cell below loads the style of this notebook. 

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../../styles/numericalmoocstyle.css'
HTML(open(css_file, "r").read())